## Imports

In [1]:
import numpy as np
import pandas as pd
import datetime
import math
import random

pd.options.display.max_columns = 50

In [2]:
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [58]:
import pprint
from collections import defaultdict
from gensim import corpora, models, similarities
# NLTK Stop words
from nltk.corpus import stopwords


## Functions

In [231]:
def process_corpus(text_corpus):
    '''
    '''
    # Create a set of frequent words
#     stoplist = ""
#     stoplist = set('for a of the and to in is our with we that by their through as\
#                    are mission on'.split(' '))
    
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_',
                   'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 
                   'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 
                   'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
    
    
    # Lowercase each document, split it by white space and filter out stopwords
    texts = ""
    texts = [[word for word in document.lower().split() if word not in stop_words] for document in text_corpus]
    
    # Count word frequencies
    frequency = defaultdict(int)
    
    for text in texts:
        for token in text:
            frequency[token] += 1
    
    words_df = pd.DataFrame(list(frequency.items()), columns= ['Words','count']).sort_values('count',ascending=False)
    
    ten_percent_cutoff = (int(len(text_corpus)*0.1))
            
    # Only keep words that appear more than once, are in less than 10% of documents, and are alphabetical strings
    processed_corpus = []
    
    for text in texts:
        token_list = []
        for token in text:
            if frequency[token] > 1 and frequency[token] < ten_percent_cutoff and str.isalpha(token):
                token_list.append(token)
        processed_corpus.append(token_list)
    
    return processed_corpus

In [232]:
def create_index_from_corpus(processed_corpus):
    '''
    '''
    dictionary = corpora.Dictionary(processed_corpus)
    features = (len(dictionary))
    
    bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
    
    # train the model
    tfidf = models.TfidfModel(bow_corpus)
    
    index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=features)
    
    return index, dictionary, tfidf

In [302]:
def find_similar_charities(train_df,test_df):
    '''
    '''
    total = 0
    category_counter = {1:0,2:0,3:0}
    document_min_words_cutoff = 200
    
    print("1. Processing Training Corpus")
    char_desc_trimmed = []
    for doc in train_df['description']:
        if len(doc) >= document_min_words_cutoff:
            char_desc_trimmed.append(doc)
    
    char_desc_trimmed = np.array(char_desc_trimmed)
    
    corpus = char_desc_trimmed
    processed_corpus = process_corpus(corpus)
    
    print("2. Creating Index from Training Corpus")
    index, dictionary, tfidf = create_index_from_corpus(processed_corpus)
    
    print("3. Starting Test Corpus Similarity Analysis\n")
    for ind, document in enumerate(test_df['description']):
        total +=1 
        if total % 500 == 0:
            print("Analyzed",total,"documents...")
        #print("Top 3 Charities Similar to:", test_df['name'].iloc[ind],'\n')
        
        query_bow = dictionary.doc2bow(document.split())
        sims = index[tfidf[query_bow]]
        
        top_3_sim = dict()
        count = 3
        
        for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
            if count > 0:
                top_3_sim[document_number] = score
            count -= 1
            
        category_count = 1
        
        for doc, score in top_3_sim.items():
            
            if train_df['category'].iloc[doc] == test_df['category'].iloc[ind]:
                category_counter[category_count] += 1
            category_count +=1
            
    # Print Scores
    
    first_rec_score = round((category_counter[1] / total)*100,2)
    second_rec_score = round((category_counter[2] / total)*100,2)
    third_rec_score = round((category_counter[3] / total)*100,2)
    
    print ("\nFirst Recommendation Score:", first_rec_score,"%")
    print ("Second Recommendation Score:", second_rec_score,"%")   
    print ("Third Recommendation Score:", third_rec_score,"%\n")
    
    print ("AVG Recommendation Score:", round((first_rec_score+second_rec_score+third_rec_score)/3,2),"%")
    
    pass
    

In [260]:
def find_similar_charities_all(train_df,test_df):
    '''
    '''
    total = 0
    category_counter = {1:0,2:0,3:0}
    document_min_words_cutoff = 200
    
    
    print("1. Processing Training Corpus")
    char_desc_trimmed = []
    for doc in train_df['description']:
        if len(doc) >= document_min_words_cutoff:
            char_desc_trimmed.append(doc)
    char_desc_trimmed = np.array(char_desc_trimmed)
    
    corpus = char_desc_trimmed
    processed_corpus = process_corpus(corpus)
    
    print("2. Creating Index from Training Corpus")
    index, dictionary, tfidf = create_index_from_corpus(processed_corpus)
    
    print("3. Starting Test Corpus Similarity Analysis\n")
    for ind, document in enumerate(test_df['description']):
        
        total +=1 
        
        if total % 500 == 0:
            print("Analyzed",total,"documents...")
        
        print(document.lower())
        
        query_bow = dictionary.doc2bow(document.lower().split())
        sims = index[tfidf[query_bow]]
        
        top_3_sim = dict()
        count = 3
        
        print(sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[:4])
        
        for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[:4]:
            
            if count > 0:
                top_3_sim[document_number] = score
            count -= 1
            
        category_count = 1
        
        for doc, score in top_3_sim.items():
            print(train_df['name'][doc])
            print(train_df['description'][doc])
            print(doc, score, "\n")
            
            # print (charity_navigator_df['category'][doc])
            # print (test_df['category'].iloc[ind])
            
            
            if train_df['category'][doc] == test_df['category'].iloc[ind]:
                category_counter[category_count] += 1
            category_count +=1
            
    # Print Scores
    
    first_rec_score = round((category_counter[1] / total)*100,2)
    second_rec_score = round((category_counter[2] / total)*100,2)
    third_rec_score = round((category_counter[3] / total)*100,2)
    
    print ("\nFirst Recommendation Score:", first_rec_score,"%")
    print ("Second Recommendation Score:", second_rec_score,"%")   
    print ("Third Recommendation Score:", third_rec_score,"%\n")
    
    print ("AVG Recommendation Score:", round((first_rec_score+second_rec_score+third_rec_score)/3,2),"%")
    
    pass
    

In [401]:
def find_similar_charities_combined(train_df,test_df):
    '''
    '''
    total = 0
    category_counter = {1:0,2:0,3:0}
    document_min_words_cutoff = 200
    
    print("1. Processing Training Corpus")
#     char_desc_trimmed = []
    
#     for doc in train_df['corpus']:
#         if len(doc) >= document_min_words_cutoff:
#             char_desc_trimmed.append(doc)
    
    char_desc_trimmed = np.array(train_df['corpus'])
    
    corpus = char_desc_trimmed
    processed_corpus = process_corpus(corpus)
    
    print("2. Creating Index from Training Corpus")
    index, dictionary, tfidf = create_index_from_corpus(processed_corpus)
    
    print("3. Starting Test Corpus Similarity Analysis\n")
    for ind, document in enumerate(test_df['corpus']):
        total +=1 
        if total % 500 == 0:
            print("Analyzed",total,"documents...")
        
        
        query_bow = dictionary.doc2bow(document.split())
        sims = index[tfidf[query_bow]]
        
        top_3_sim = dict()
        count = 3
        
        for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[1:4]:
            
            if count > 0:
                top_3_sim[document_number] = score
            count -= 1
            
        category_count = 1
        
        print("Top 3 Charities Similar to:\nName:", test_df['name'].iloc[ind])
        print('Category:', test_df['category'].iloc[ind],'\n')
        
        for doc, score in top_3_sim.items():
            print("Name:",train_df['name'].iloc[doc])
            print("Category:", train_df['category'].iloc[doc])
            print("Score:",score)
            
            if train_df['category'].iloc[doc] == test_df['category'].iloc[ind]:
                category_counter[category_count] += 1
            category_count +=1
        print("")
            
    # Print Scores
    
    first_rec_score = round((category_counter[1] / total)*100,2)
    second_rec_score = round((category_counter[2] / total)*100,2)
    third_rec_score = round((category_counter[3] / total)*100,2)
    
    print ("\nFirst Recommendation Score:", first_rec_score,"%")
    print ("Second Recommendation Score:", second_rec_score,"%")   
    print ("Third Recommendation Score:", third_rec_score,"%\n")
    
    print ("AVG Recommendation Score:", round((first_rec_score+second_rec_score+third_rec_score)/3,2),"%")
    
    return top_3_sim
    

## Loading Charity Navigator Data

In [7]:
charity_navigator_df = pd.read_csv('../data/CLEAN_charity_data.csv')
charity_navigator_df = charity_navigator_df[['name','ein','category','description','motto','score','state']]
charity_navigator_df['ein'] = charity_navigator_df['ein'].apply(lambda x: int(x.replace("-","")))

In [8]:
charity_navigator_df.head()

,name,ein,category,description,motto,score,state
0,1000 Friends of Oregon,930642086,Environment,Working with Oregonians to enhance our quality...,Great communities. Working lands. Iconic Places.,91.94,OR
1,WYPR,311770828,"Arts, Culture, Humanities",Serving the metropolitan Baltimore area and th...,88.1 FM -. Your NPR News Station,85.59,MD
2,VSS Catholic Communications,911857425,Religion,VSS Catholic Communications is dedicated to an...,Spirit Catholic Radio Network,76.80,NE
3,Utah Symphony & Opera,510145980,"Arts, Culture, Humanities",The mission of the Utah Symphony & Opera is to...,"Engaging, educating, and enriching lives",91.95,UT
4,Two Ten Footwear Foundation,222579809,Human Services,"Funded solely by the footwear industry, Two Te...",Shoepeople Helping Shoepeople,90.26,MA


In [238]:
charity_navigator_df.groupby('category').size()

category
Animals                        454
Arts, Culture, Humanities     1218
Community Development          803
Education                      667
Environment                    429
Health                         847
Human Services                2379
Human and Civil Rights         346
International                  610
Religion                       450
Research and Public Policy     205
dtype: int64

In [10]:
# There are 11 Categories Total
## A "Random Guess" Baseline is 1/11 = 9.09%

## Modeling

In [11]:
train_df = charity_navigator_df[:6000]
char_desc_trimmed = []
for doc in train_df['description']:
    if len(doc) >= 200:
        char_desc_trimmed.append(doc)
char_desc_trimmed = np.array(char_desc_trimmed)

In [12]:
len(char_desc_trimmed)

5616

### Sequential Train/Test Split

#### Best Scores:

1. 17.23%
2. 16.07%
3. 16.03%

#### Total = 16.44%

In [239]:
train_df = charity_navigator_df[:6000]
test_df = charity_navigator_df[6000:]

find_similar_charities(train_df,test_df)

1. Processing Training Corpus
2. Creating Index from Training Corpus
3. Starting Test Corpus Similarity Analysis

Analyzed 500 documents...
Analyzed 1000 documents...
Analyzed 1500 documents...
Analyzed 2000 documents...

First Recommendation Score: 14.78 %
Second Recommendation Score: 16.78 %
Third Recommendation Score: 16.9 %

AVG Recommendation Score: 16.15 %


### Random Train/Test Split (Test Size = 30%)

#### Best Scores:

1. 14.70%
2. 14.15%
3. 15.74%

#### Total = 14.86%

In [230]:
test_df['category'].iloc[1695]

'Education'

In [234]:
train_df, test_df = train_test_split(charity_navigator_df,test_size = 0.30)

find_similar_charities(train_df,test_df)

1. Processing Training Corpus
2. Creating Index from Training Corpus
3. Starting Test Corpus Similarity Analysis

Analyzed 500 documents...
Analyzed 1000 documents...
Analyzed 1500 documents...
Analyzed 2000 documents...
Analyzed 2500 documents...

First Recommendation Score: 14.51 %
Second Recommendation Score: 14.15 %
Third Recommendation Score: 14.15 %

AVG Recommendation Score: 14.27 %


### Random Train/Test Split (Test Size = 25%)

#### Best Scores:

1. 14.37%
2. 14.13%
3. 14.84%

#### Total = 14.45%

In [236]:
train_df, test_df = train_test_split(charity_navigator_df,test_size = 0.25)

find_similar_charities(train_df,test_df)

1. Processing Training Corpus
2. Creating Index from Training Corpus
3. Starting Test Corpus Similarity Analysis

Analyzed 500 documents...
Analyzed 1000 documents...
Analyzed 1500 documents...
Analyzed 2000 documents...

First Recommendation Score: 14.22 %
Second Recommendation Score: 15.46 %
Third Recommendation Score: 15.22 %

AVG Recommendation Score: 14.97 %


### Random Train/Test Split (Test Size = 20%)

#### Best Scores:

1. 14.09%
2. 15.40%
3. 12.78%

#### Total = 14.09%

In [237]:
train_df, test_df = train_test_split(charity_navigator_df,test_size = 0.20)

find_similar_charities(train_df,test_df)

1. Processing Training Corpus
2. Creating Index from Training Corpus
3. Starting Test Corpus Similarity Analysis

Analyzed 500 documents...
Analyzed 1000 documents...
Analyzed 1500 documents...

First Recommendation Score: 15.87 %
Second Recommendation Score: 17.0 %
Third Recommendation Score: 14.33 %

AVG Recommendation Score: 15.73 %


### Train on full data set, test on individual charities

#### Best Scores:

1. 14.09%
2. 15.40%
3. 12.78%

#### Total = 14.09%

In [75]:
find_similar_charities(charity_navigator_df,charity_navigator_df.loc[19:50])

1. Processing Training Corpus
2. Creating Index from Training Corpus
3. Starting Test Corpus Similarity Analysis


First Recommendation Score: 12.5 %
Second Recommendation Score: 21.88 %
Third Recommendation Score: 9.38 %

AVG Recommendation Score: 14.59 %


In [81]:
print(len(charity_navigator_df))

charity_navigator_df.iloc[[19,30]]

8408


,name,ein,category,description,motto,score,state
19,Zoo Miami Foundation,596192814,Animals,Zoo Miami Foundation (ZMF) is the non-profit s...,Our mission is to foster community pride and i...,93.74,FL
30,Issaquah Schools Foundation,943050254,Education,"Founded in 1987, in cooperation with the Issaq...",Helping students succeed,86.14,WA


In [270]:
charity_navigator_df.iloc[[70]]['category']

70    Animals
Name: category, dtype: object

In [ ]:
find_similar_charities_all(charity_navigator_df, charity_navigator_df.iloc[[100]])

In [353]:
temp_df = charity_navigator_df.copy()

In [363]:
temp_df['motto'] = temp_df['motto'].fillna(" ")

In [364]:
temp_df.isna().sum()

name           0
ein            0
category       0
description    0
motto          0
score          0
state          0
corpus         0
dtype: int64

In [ ]:
temp_df

In [365]:
temp_df['corpus'] = (temp_df['category'] + " " + temp_df['description'] + " " +temp_df['motto']+" "+ temp_df['state'])

In [393]:
temp_df.to_csv('../data/charities_rated_corpus.csv')

In [378]:
train_df, test_df = train_test_split(temp_df,test_size = 0.20)

find_similar_charities_combined(train_df,test_df)

1. Processing Training Corpus
2. Creating Index from Training Corpus
3. Starting Test Corpus Similarity Analysis

Top 3 Charities Similar to:
Name: United Way of Anderson County
Category: Community Development 

Name: United Way of Eastern Maine
Category: Community Development
Name: United Way of East Central Alabama
Category: Community Development
Name: United Way Worldwide
Category: Community Development

Top 3 Charities Similar to:
Name: New England Foundation for the Arts
Category: Arts, Culture, Humanities 

Name: Connect2Help 2-1-1
Category: Human Services
Name: Coastal Maine Botanical Gardens
Category: Environment
Name: Boys & Girls Aid
Category: Human Services

Top 3 Charities Similar to:
Name: Connecticut Association for Human Services
Category: Research and Public Policy 

Name: Denver Zoological Foundation
Category: Animals
Name: My Sister's Place
Category: Human Services
Name: Futures Without Violence
Category: Human and Civil Rights

Top 3 Charities Similar to:
Name: Ventu

Name: Chicago Humanities Festival
Category: Arts, Culture, Humanities 

Name: Interfaith Outreach & Community Partners
Category: Human Services
Name: World Institute on Disability
Category: Research and Public Policy
Name: Sci-Port
Category: Arts, Culture, Humanities

Top 3 Charities Similar to:
Name: The Education Foundation of Collier County
Category: Education 

Name: Teach Plus Incorporated
Category: Education
Name: Mid-America Science Museum
Category: Arts, Culture, Humanities
Name: New Teacher Center
Category: Education

Top 3 Charities Similar to:
Name: United Way of Southern Chautauqua County
Category: Community Development 

Name: United Way of Cascade County
Category: Community Development
Name: United Way of Northeast Georgia
Category: Community Development
Name: United Way of Pioneer Valley
Category: Community Development

Top 3 Charities Similar to:
Name: Alpha USA
Category: Religion 

Name: Eyebeam
Category: Arts, Culture, Humanities
Name: Science Center of Iowa
Category:

Category: Community Development 

Name: Austin Community Foundation
Category: Community Development
Name: SMART
Category: Education
Name: Miami Foundation
Category: Community Development

Top 3 Charities Similar to:
Name: Forsyth Humane Society
Category: Animals 

Name: Humane Society for Southwest Washington
Category: Animals
Name: League for Animal Welfare
Category: Animals
Name: Animal Haven
Category: Animals

Top 3 Charities Similar to:
Name: Glimmerglass Festival
Category: Arts, Culture, Humanities 

Name: Cincinnati Opera
Category: Arts, Culture, Humanities
Name: Santa Barbara Opera Association
Category: Arts, Culture, Humanities
Name: Michigan Opera Theatre
Category: Arts, Culture, Humanities

Top 3 Charities Similar to:
Name: The Gentle Barn
Category: Animals 

Name: The Wild Animal Sanctuary
Category: Animals
Name: Teach For America
Category: Education
Name: Council for Economic Education
Category: Education

Top 3 Charities Similar to:
Name: American Jewish Congress
Category:

Name: Orbis International
Category: International 

Name: Prevent Blindness
Category: Health
Name: Seva Foundation
Category: International
Name: Kempe Foundation for the Prevention and Treatment of Child Abuse and Neglect
Category: Human Services

Top 3 Charities Similar to:
Name: Depression and Bipolar Support Alliance, National Office
Category: Health 

Name: American Foundation for Suicide Prevention
Category: Health
Name: The Jed Foundation
Category: Health
Name: The Trevor Project
Category: Human and Civil Rights

Top 3 Charities Similar to:
Name: World Emergency Relief
Category: International 

Name: All About Developmental Disabilities
Category: Human Services
Name: Help the Children
Category: Human Services
Name: Helping Hand for Relief and Development
Category: International

Top 3 Charities Similar to:
Name: Water Street Ministries
Category: Human Services 

Name: Kingdom Workers
Category: Religion
Name: Cary Christian Center
Category: Human Services
Name: Light of Life Rescu

Name: Northampton Survival Center
Category: Human Services 

Name: God's Pantry Food Bank
Category: Human Services
Name: The Food Bank for Central & Northeast Missouri
Category: Human Services
Name: Food for Others
Category: Human Services

Top 3 Charities Similar to:
Name: Big Brothers Big Sisters of South Texas
Category: Human Services 

Name: Big Brothers Big Sisters of the Greater Twin Cities
Category: Human Services
Name: Big Brothers Big Sisters of Miami
Category: Human Services
Name: Big Brothers Big Sisters of Tampa Bay
Category: Human Services

Top 3 Charities Similar to:
Name: Texas Civil Rights Project
Category: Human and Civil Rights 

Name: Michigan League for Public Policy
Category: Research and Public Policy
Name: Southeastern Legal Foundation
Category: Human and Civil Rights
Name: Public Advocates
Category: Human and Civil Rights

Top 3 Charities Similar to:
Name: Urban League of Eastern Massachusetts
Category: Human and Civil Rights 

Name: Animal Rescue League of Bost

...

Name: Contemporary Art Museum St. Louis
Category: Arts, Culture, Humanities 

Name: Museum of Contemporary Art, Chicago
Category: Arts, Culture, Humanities
Name: Contemporary Arts Center, Cincinnati
Category: Arts, Culture, Humanities
Name: Virginia Museum of Contemporary Art
Category: Arts, Culture, Humanities

Top 3 Charities Similar to:
Name: Covenant House Michigan
Category: Human Services 

Name: Covenant House Alaska
Category: Human Services
Name: Saban Community Clinic
Category: Health
Name: Family Resources
Category: Human Services

Top 3 Charities Similar to:
Name: Channel 3 Kids Camp
Category: Education 

Name: Camp Quality USA
Category: Health
Name: One Heartland
Category: Health
Name: Camp Smile-A-Mile
Category: Health

Top 3 Charities Similar to:
Name: Marine Corps University Foundation
Category: Education 

Name: Blue Star Families
Category: Human Services
Name: Military Community Youth Ministries
Category: Religion
Name: Operation Homefront
Category: Human Services

Top 

Top 3 Charities Similar to:
Name: University City Children's Center
Category: Human Services 

Name: Sheltering Arms Early Education & Family Centers
Category: Education
Name: Vogel Alcove
Category: Human Services
Name: Think Small
Category: Human Services

Top 3 Charities Similar to:
Name: Grantmakers in the Arts
Category: Arts, Culture, Humanities 

Name: Foundation for Contemporary Arts
Category: Arts, Culture, Humanities
Name: Center for International Policy
Category: International
Name: The Climate Reality Project
Category: Environment

Top 3 Charities Similar to:
Name: Mid-Atlantic Foundation for Safety and Education
Category: Research and Public Policy 

Name: Test Positive Awareness Network
Category: Health
Name: California Highway Patrol 11-99 Foundation
Category: Human Services
Name: American Council on Science and Health
Category: Research and Public Policy

Top 3 Charities Similar to:
Name: Lymphoma Research Foundation
Category: Health 

Name: Huntington's Disease Society o

Top 3 Charities Similar to:
Name: United Way of South Hampton Roads
Category: Community Development 

Name: United Way of West Tennessee
Category: Community Development
Name: United Way of Ponca City
Category: Community Development
Name: Orange County United Way
Category: Community Development

Top 3 Charities Similar to:
Name: Rainforest Alliance
Category: Environment 

Name: National Consumers League
Category: Research and Public Policy
Name: Wildlife Alliance
Category: Animals
Name: Global Communities
Category: International

Top 3 Charities Similar to:
Name: Greater Hartford Arts Council
Category: Arts, Culture, Humanities 

Name: National Review Institute
Category: Arts, Culture, Humanities
Name: Bottom Line
Category: Education
Name: World Education
Category: International

Top 3 Charities Similar to:
Name: Issaquah Schools Foundation
Category: Education 

Name: First Place
Category: Education
Name: United Jewish Federation of Greater Stamford, New Canaan and Darien
Category: Comm

Top 3 Charities Similar to:
Name: Family Assistance Ministries
Category: Human Services 

Name: Discovery Eye Foundation
Category: Health
Name: OUR Center
Category: Human Services
Name: Sankara Eye Foundation, USA
Category: Health

Top 3 Charities Similar to:
Name: Pioneer Institute
Category: Research and Public Policy 

Name: Government Accountability Project
Category: Human and Civil Rights
Name: Coalition for Engaged Education
Category: Education
Name: The Mockingbird Society
Category: Human Services

Top 3 Charities Similar to:
Name: Ronald McDonald House Charities of Madison
Category: Health 

Name: Tri-County Community Dental Clinic
Category: Health
Name: Ronald McDonald House Charities of Rochester
Category: Health
Name: Interfaith Dental Clinic
Category: Health

Top 3 Charities Similar to:
Name: United Way of Greater Greensboro
Category: Community Development 

Name: United Way of Kosciusko County
Category: Community Development
Name: United Way of St. Lucie County
Category: Co

Name: Water.org
Category: International 

Name: Water For People
Category: International
Name: WaterAid America
Category: International
Name: Water Mission
Category: International

Top 3 Charities Similar to:
Name: Bigelow Laboratory for Ocean Sciences
Category: Research and Public Policy 

Name: The Ocean Foundation
Category: Environment
Name: Ocean Discovery Institute
Category: Education
Name: Loggerhead Marinelife Center
Category: Animals

Top 3 Charities Similar to:
Name: Institute for Transportation and Development Policy
Category: Environment 

Name: Environmental and Energy Study Institute
Category: Environment
Name: Outreach International
Category: International
Name: Center for Creative Land Recycling
Category: Environment

Top 3 Charities Similar to:
Name: Wildlife Conservation Society
Category: Animals 

Name: National Wildlife Federation
Category: Animals
Name: Wildlife Center of Virginia
Category: Animals
Name: Wildlife Forever
Category: Animals

Top 3 Charities Similar to

In [402]:
top_3_sim = find_similar_charities_combined(temp_df,temp_df[temp_df['name']=='National Wildlife Federation'])

1. Processing Training Corpus
2. Creating Index from Training Corpus
3. Starting Test Corpus Similarity Analysis

Top 3 Charities Similar to:
Name: National Wildlife Federation
Category: Animals 

Name: Wildlife Center of Virginia
Category: Animals
Score: 0.4614803
Name: Zoo Atlanta
Category: Animals
Score: 0.4182947
Name: Audubon Society of Portland
Category: Environment
Score: 0.39689428


First Recommendation Score: 100.0 %
Second Recommendation Score: 100.0 %
Third Recommendation Score: 0.0 %

AVG Recommendation Score: 66.67 %


In [403]:
top_3_sim

{4357: 0.4614803, 20: 0.4182947, 4535: 0.39689428}

In [416]:
file_info = temp_df.iloc[2357:2358]

In [423]:
file_info.name.iloc[0]

'Pet Helpers'

In [400]:
temp_df[temp_df['name']=='National Wildlife Federation']

,name,ein,category,description,motto,score,state,corpus
1698,National Wildlife Federation,530204616,Animals,National Wildlife Federation's (NWF) mission i...,Inspiring Americans to protect wildlife for ou...,81.19,VA,Animals National Wildlife Federation's (NWF) m...


In [ ]:
National Wildlife Federation